## Rental Registration Spreadsheet Cleanup  
There are several types of error that make the raw spreadsheet unusable for joins with other data sets.  All of them come down to problems finding a sutable string in the join key 'acctid', which is a normalized form of what the state calls the 'district' and 'account identifier' fields.  

acctid = '100' + \<district\> + \<account identifier\>  
 
The problems are tyically that the necessary fields is missing, the fields a mis-entered, and sometimes there are duplicate rows that cause confusion.  This notebook provides tools for cleaning up the rental registrations so there is a usable 'acctid' field.  

The methodology here is to accumulate corrections in a file that can be applied to correct the data.  That way we can accumulate fixes in this notebook, and have a file we can use to record fixes that we have to resolve manually (looking in SDAT, checking a map, etc).

There is also code at the bottom of the sheet for clustering properties by owner information pulled from SDAT.

In [49]:
!pip install simpledbf

     |████████████████████████████████| 163kB 6.8MB/s 
  Created wheel for leven: filename=leven-1.0.4-cp37-cp37m-linux_x86_64.whl size=55483 sha256=2064ccf24021b85aecbf915a446d206cbb969719a2f82609d5558be1d59aa8c7
  Stored in directory: /root/.cache/pip/wheels/54/64/a5/439db671d666a50f3b3cebd2dcab3fbbab02785adf58e47552
Successfully built leven


In [4]:
import pandas as pd
from simpledbf import Dbf5
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Load the rental billing spreadsheet to clean up

In [6]:
#df = pd.read_csv('/content/drive/My Drive/pita 2021/rental billing 2019.csv')
df = pd.read_csv('/content/drive/My Drive/pita 2021/rental billing 2019.csv')

# strip off rows with comments, etc
print('Input column headers')
print(df.columns)
print('raw rows:',len(df))

# clean up the column headers
df.rename(columns={'Dist/Account No    ':'Dist/Account No','RENTAL  ':'RENTAL'},inplace=True)
df = df[df['RENTAL'].notna()]

# fix up account numbers to make apn-format tax acctid column
df['Dist/Account No'] = df['Dist/Account No'].fillna('-1')
df['Dist/Account No'] = df.apply(lambda x: x['Dist/Account No'].replace(' ',''),axis=1)
df['acctid'] = df.apply(lambda x: "10{}".format(x['Dist/Account No'].replace('-','')), axis = 1)

print('useful rows:',len(df))
print('Updated column headers')
print(df.columns)


Input column headers
Index(['License Id', 'License Type Id', 'Business Name', 'Customer Id',
       'Effective Date', 'Expiration Date', 'State Id', 'Status',
       'Property Location', 'Dist/Account No    ', 'RENTAL  ', 'Unnamed: 11',
       'Unnamed: 12'],
      dtype='object')
raw rows: 1556
useful rows: 1550
Updated column headers
Index(['License Id', 'License Type Id', 'Business Name', 'Customer Id',
       'Effective Date', 'Expiration Date', 'State Id', 'Status',
       'Property Location', 'Dist/Account No', 'RENTAL', 'Unnamed: 11',
       'Unnamed: 12', 'acctid'],
      dtype='object')


#### Load the latest SDAT information, and join it

In [21]:
sdat = pd.read_csv('drive/My Drive/pita 2021/SDAT-CAN-ref-202105.csv')
sdat.acctid = sdat.acctid.apply(lambda x: str(x).strip())
sdat = sdat.set_index('acctid')
sdat

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,11,12,13,14,15,16,29,38,45,49,52,53,77,82,86,104,106,112,113,129,137) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,jurscode,digxcord,digycord,ct2010,bg2010,geogcode,ooi,resityp,address,strtnum,strtdir,strtnam,strttyp,strtsfx,strtunt,addrtyp,city,zipcode,ownname1,ownname2,namekey,ownadd1,ownadd2,owncity,ownstate,ownerzip,ownzip2,premsnum,premsdir,premsnam,premstyp,premcity,premzip,premzip2,legal1,legal2,legal3,dr1clerk,dr1liber,dr1folio,...,crtarcod,fcmacode,agfndarea,agfndluom,entzndat,entznassm,plndevdat,nprctstdat,nprcarea,nprcluom,homqlcod,homqldat,bldg_story,bldg_units,resident,resi2010,resi2000,resi1990,resiuths,aprtment,trailer,special,other,ptype,sdatwebadr,existing,mdpvdate,sdat,google_maps,struct_sqft,assessed_value,address_number,address_unit_id,street_direction,street_name,street_type,premise_address_type_mdp_field_premstyp_sdat_field_24,premise_address_city_mdp_field_premcity_sdat_field_25,premise_address_zip_code_mdp_field_premzip_sdat_field_26,mdp_street_address_mdp_field_address
acctid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001000012,DORC,508948.5,110654.0,2.401997e+10,2.401997e+11,80,N,SF,5727 ADAMS ROAD,5727.0,NaN,ADAMS,RD,NaN,NaN,P,FEDERALSBURG,21632.0,NAGEL RICHARD LEE & CONNIE JANE,NaN,NAGEL RICHARD LEE & CONNI,5714 ADAMS RD,NaN,FEDERALSBURG,MD,21632.0,1700.0,5727.0,NaN,ADAMS,RD,FEDERALSBURG,21632.0,NaN,52.94 ACRES,S/S ADAMS RD.,NE OF FINCHVILLE,MLB,363.0,779.0,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN,http://sdat.dat.maryland.gov/RealProperty/Page...,https://maps.google.com/maps?t=h&q=38.65678537...,0,22100,0,5727,NaN,NaN,ADAMS,RD,FEDERALSBURG,21632.0,5727 ADAMS ROAD
1001000039,DORC,511216.1,106713.9,2.401997e+10,2.401997e+11,80,H,SF,6009 COKESBURY ROAD,6009.0,NaN,COKESBURY,RD,NaN,NaN,P,RHODESDALE,19973.0,GARDINER KEVIN E,GARDINER LORI A,GARDINER KEVIN E,6009 COKESBURY RD,NaN,SEAFORD,DE,19973.0,NaN,6009.0,NaN,COKESBURY,RD,SEAFORD,19973.0,NaN,IMPS4.80 ACRES,E/S COKESBURY RD,SW/RELIANCE,NaN,1493.0,455.0,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN,http://sdat.dat.maryland.gov/RealProperty/Page...,https://maps.google.com/maps?t=h&q=38.62101071...,3492,287100,2004,6009,NaN,NaN,COKESBURY,RD,SEAFORD,19973.0,6009 COKESBURY ROAD
1001000047,DORC,508807.3,110360.1,2.401997e+10,2.401997e+11,80,N,SF,5731 DAVIS MILL POND ROAD,5731.0,NaN,DAVIS MILL POND,RD,NaN,NaN,P,FEDERALSBURG,21632.0,HARIM MILLSBORO LLC,NaN,HARIM MILLSBORO LLC,PO BOX 1380,MAILSTOP 100484,MILLSBORO,DE,19966.0,NaN,5731.0,NaN,DAVIS MILL POND,RD,NaN,NaN,NaN,IMPS20 ACRES,W/S DAVIS MILLPOND RD,NE/FINCHVILLE,NaN,1471.0,11.0,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN,http://sdat.dat.maryland.gov/RealProperty/Page...,https://maps.google.com/maps?t=h&q=38.65415540...,1438,90400,1920,5731,NaN,NaN,DAVIS MILL POND,RD,NaN,0.0,5731 DAVIS MILL POND ROAD
1001000055,DORC,507295.0,112993.8,2.401997e+10,2.401997e+11,80,N,TR,6940 RELIANCE ROAD,6940.0,NaN,RELIANCE,RD,NaN,NaN,P,FEDERALSBURG,21632.0,HARIM MILLSBORO LLC,NaN,HARIM MILLSBORO LLC,PO BOX 1380,MAILSTOP 100484,MILLSBORO,DE,19966.0,NaN,6940.0,NaN,RELIANCE,RD,NaN,NaN,NaN,IMPS232 AC,S/W ALLENS COR-FDG RD,W/ALLENS COR,NaN,1471.0,11.0,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,http://sdat.dat.maryland.gov/RealProperty/Page...,MDPV2017_18,2020JUN,http://sdat.dat.maryland.gov/RealProperty/Page...,https://maps.google.com/maps?t=h&q=38.67806461...,1056,181000,1989,6940,NaN,NaN,RELIANCE,RD,NaN,0.0,6940 RELIANCE ROAD
1001000063,DORC,511793.2,105251.7,2.401997e+10,2.401997e+11,80,D,SF,6002 ALLEN ROAD,6002.0,NaN,ALLEN,RD,NaN,NaN,P,RHODESDALE,19973.0,DONOVAN MICHAEL,DONOVAN VICKI,DONOVAN MICHAEL,6002 ALLEN RD,RT 3 BOX 270,SEAFORD,DE,19973.0,6057.0,6002.0,NaN,ALLEN,RD,S

In [22]:
sdat_merge_df = df.merge(sdat,left_on='acctid',right_on='acctid',how='outer',indicator=True)
cleaned_registrations_df = sdat_merge_df.query('_merge == "both"')[list(df.columns)+['address']]
print('found:',len(cleaned_registrations_df))
print('problem records, in the rental sheet but not in sdat?:',len(sdat_merge_df.query('_merge == "left_only"')))
cleaned_registrations_df

found: 1491
problem records, in the rental sheet but not in sdat?: 59


,License Id,License Type Id,Business Name,Customer Id,Effective Date,Expiration Date,State Id,Status,Property Location,Dist/Account No,RENTAL,Unnamed: 11,Unnamed: 12,acctid,address
0,15-00183,RENTAL,DEREK MILLER,RR-01223,7/1/2019,6/30/2020,,Approved,406 KENT ST 4144,07-143230,1.0,True,True,1007143230,406 KENT ST
1,18-01619,RENTAL,BRYAN JEFFREY & JULIE,RR-96791,7/1/2019,6/30/2020,,Approved,700 CATTAIL COVE UNIT# 301,07-213905,1.0,True,True,1007213905,700 CATTAIL COVE
2,19-00001,RENTAL,OTTER LLC,RR-07388,7/1/2019,6/30/2020,,Approved,416 BOUNDARY AVE 128,07-113935,1.0,True,True,1007113935,416 BOUNDARY AVE
3,19-00002,RENTAL,DAGOSTINO COREY,RR-07981,7/1/2019,6/30/2020,,Approved,704 CHURCH ST 374,07-148038,2.0,True,True,1007148038,704 CHURCH ST
4,19-00003,RENTAL,MILLER DURELL,RR-03640,7/1/2019,6/30/2020,,Approved,623 DOUGLAS ST 471,07-164289,1.0,True,True,1007164289,623 DOUGLAS ST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545,19-01523,RENTAL,KLEYMAN ALEKANDAR,RR-09679,7/1/2019,6/30/2020,,Approved,900 MARSHY COVE UNIT# 302,07-214987,1.0,True,True,1007214987,900 MARSHY COVE
1546,19-01524,RENTAL,BRADLEY THOMAS,RR-06434,7/1/2019,6/30/2020,,Approved,1000 RACE ST,07-165730,1.0,True,True,1007165730,1000 RACE ST
1547,19-01543,RENTAL,ADAMS JOEL,RR-04294,7/1/2019,6/30/2020,,Approved,811 PINE ST 1528,07-152078,1.0,True,True,1007152078,811 PINE ST
1548,19-01546,RENTAL,EHISAM FREDERICK,RR-09678,7/1/2019,6/30/2020,,Approved,900 MARSHY COVE UNIT# 312,07-215096,1.0,True,True,1007215096,900 MARSHY COVE


### Fix up any problem records. 
The next few cells are ways to check for errors.  
The first method is to take the prior work, and try to match fields picked up from this process last time.

In [23]:
# start with the left join of the prior result, that's the records that didn't match
problems_df = sdat_merge_df.query('_merge == "left_only"')

# grab the acctid from a prior year where you can, and try to merge with sdat using that key for some of the bad rows
history_df = pd.read_csv('drive/My Drive/pita 2021/cambridge-combined-old-new-rental-lists-17-18.csv').rename(columns={'ACCTID':'acctid'})
history_df = history_df[history_df['acctid'].notna()]
history_df.acctid = history_df.acctid.apply(lambda x: str(x).strip())
fixups_df = problems_df.drop(columns=['_merge']).merge(history_df,on='acctid',how='outer',indicator=True).drop_duplicates()

print("these can be fixed leverging prior results:", len(fixups_df[(fixups_df['_merge'] == "both")]))
cleaned_registrations_df = cleaned_registrations_df.append(fixups_df[(fixups_df['_merge'] == "both")][list(df.columns)+['address']])
print(len(cleaned_registrations_df),"of",len(df),"rows cleaned")
print("these still need more work:",len(fixups_df[(fixups_df['_merge'] == "left_only")] ))

these can be fixed leverging prior results: 9
1500 of 1550 rows cleaned
these still need more work: 51


#### One thing to do is to try a join on address...  
but first you have to clean them up a bit

In [24]:
def cleanup_address(a):
  b = a.strip().replace('.','').replace('AVENUE','AVE')
  pieces = b.split()
  if pieces[-1][0].isnumeric():
    return " ".join(pieces[0:-1]).replace('.','')
  else:
    return b

corrected_address_df = fixups_df[(fixups_df['_merge'] == "left_only")].drop(columns='_merge')
corrected_address_df['Property Location'] = corrected_address_df.apply(lambda x: cleanup_address( x['Property Location'] ), axis=1)

In [41]:
# these are found
corrected_address_join = corrected_address_df.drop(columns=['acctid','address']).merge(sdat.reset_index()[['acctid','address']],
                              left_on='Property Location',right_on='address',
                              how='outer',indicator=True)
found_by_join_on_address = corrected_address_join.query('_merge == "both"')[df.columns.to_list()+['address']]

cleaned_registrations_df = cleaned_registrations_df.append(found_by_join_on_address[list(df.columns)+['address']])
print(len(cleaned_registrations_df),"of",len(df),"rows cleaned")
print("these still need more work:",len(corrected_address_join.query('_merge == "left_only"')[df.columns.to_list()+['address']]))

1568 of 1550 rows cleaned
these still need more work: 17


In [ ]:
# the easiest thing to do may be to fix thees in the original file

corrected_address_df[['Property Location']].merge(sdat[['address','acctid']],left_on='Property Location',right_on='address',how='outer',indicator=True).query('_merge == "left_only"')

,Property Location,address,acctid,_merge
4,701 MOORES AVE,NaN,NaN,left_only
13,700 CATTAIL COVE UNIT 110,NaN,NaN,left_only
17,712 LINCOLN TERRACE,NaN,NaN,left_only
20,303 BAYLY RD,NaN,NaN,left_only
22,1403 STONE BOUNDARY RD,NaN,NaN,left_only
25,5242 GALLIUM CT,NaN,NaN,left_only
32,203 ROBBINS ST APT A,NaN,NaN,left_only
33,105 RAMBLER RD,NaN,NaN,left_only
34,707 PEACHBLOSSOM AVE DOWN,NaN,NaN,left_only
36,116 BLACK DUCK DR,NaN,NaN,left_only


## WRITE OUT THE CLEANED RENTAL BILLING  
Any remaining above will need to be added by hand?

In [ ]:
cleaned_registrations_df.reset_index(drop=True).to_csv('/content/drive/My Drive/pita 2021/cleaned_rental_billing-2020-test.csv')

## ADD Fixups  
This section adds back things accumulated in a separate csv file as fixups.

As more items in the list are resolved, the fixup list will shrink at each update.  The method here, of merging historical data with the cleanded spresdsheet will resolve most issues in the future.

In [51]:
# TODO

# # rentals = df.merge(license_df,on='acctid',how='outer',indicator=True)
# # rentals.query('_merge == "right_only"')

# ## NOTE, THIS DROPS THE WOODS ROAD BUILDINGS !!!!
# from collections import defaultdict 

# licensed_accts = set(license_df.acctid.array)
# rentals = df.merge(license_df,on='acctid',how='inner')#,indicator=True)
# rental_accts = set(rentals.acctid.array)
# split_accts = licensed_accts.difference(rental_accts)
# split_accts.add('1007127049-1')
# split_accts = {(acct.split("-")[0],acct.split("-")[1]) for acct in split_accts }
# acct_splits = defaultdict(list)
# for k,v in split_accts:
#   acct_splits[k].append(v)

# # make a df with the expanded reccords in it
# def add_extras(acct_splits,split=False):
#   df_extras = pd.DataFrame(columns=df.columns)
#   for acct in acct_splits.keys():
#     row = df.query('acctid == @acct').copy()
#     if split:
#       for variant in acct_splits[acct]:
#         row.acctid = acct+"-"+str(variant)
#     df_extras = df_extras.append(row)
#   return df_extras

# rentals = rentals.append(add_extras(acct_splits).merge(license_df,on='acctid',how='inner'))